In [62]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import numpy as np
import pandas as pd

In [ ]:
setup_logging()

In [108]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example')

ConfigurationError: Configuration already created!

In [ ]:
from hdx.facades.simple import facade

In [ ]:
datasets = Dataset.get_all_dataset_names()
print(len(datasets)) #Total number of datasets obtained using the API


In [82]:
datasets.index('burkina-faso-healthsites')

528

In [104]:
dataset_1 = Dataset.read_from_hdx(datasets[102]) 
resource_1 = dataset_1.get_resources() 
print(len(resource_1))
print(resource_1[0].get_file_type()) 
print(datasets[102])

1
CSV
acled-data-for-burundi


In [105]:
#Script to clean the datasets list and return the list of the resource in each dataset
# of the required file type. Does NOT return the list of the datasets themselves
#Currently very naive and inefficient [Unfinished]
def clean(datasets, file_type):
    result = []
    index = 0
    while (index < len(datasets)):
        temp_dataset = Dataset.read_from_hdx(datasets[index])
        temp_resources = temp_dataset.get_resources() 
        if (len(temp_resources) > 0):
            resource_index = 0
            while (resource_index < len(temp_resources)):
                

In [106]:
url, path = resource_1[0].download('Datasets') #Substitute Datasets with your file download path.
print('Resource URL %s downloaded to %s' % (url, path))

Resource URL https://proxy.hxlstandard.org/data/e0c1eb.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26terms%3Daccept%26iso%3D108&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&

In [110]:
resource_1[0].get_dataset().get_tags()

['conflicts', 'hxl', 'political violence', 'protests']

In [63]:
burundi_dataset = pd.read_csv(path)
burundi_dataset

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,actor1,assoc_actor_1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,NaN,#country+code,#event+code,NaN,#date+occurred,NaN,NaN,#event+type,#group+name+first,#group+name+first+assoc,...,#loc+name,#geo+lat,#geo+lon,NaN,#meta+source,NaN,#description,#affected+killed,NaN,NaN
1,3080374.0,108,BUR8585,8585.0,2019-02-11,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Gitega,-3.4264,29.9308,1.0,SOS Medias Burundi,Other,"11 February. During the night, Imbonerakure br...",0,1.550526e+09,BDI
2,3080373.0,108,BUR8584,8584.0,2019-02-10,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Kaguhu,-2.8359,30.1756,1.0,Local Source,Other,10 February. Imbonerakure beat a man to death ...,1,1.550526e+09,BDI
3,3080371.0,108,BUR8582,8582.0,2019-02-09,2019.0,2.0,Violence against civilians,Unidentified Armed Group (Burundi),NaN,...,Carama,-3.3071,29.3813,2.0,Local Source,Other,09 February (on or before). The head of an uni...,1,1.550526e+09,BDI
4,3080372.0,108,BUR8583,8583.0,2019-02-09,2019.0,1.0,Violence against civilians,Rioters (Burundi),Vigilante Group (Burundi),...,Burimbi,-2.9479,29.4034,1.0,Local Source,Other,09 February. An unidentified group set fire to...,3,1.550526e+09,BDI
5,2998715.0,108,BUR8570,8570.0,2019-02-08,2019.0,1.0,Violence against civilians,Unidentified Armed Group (Burundi),NaN,...,Nyarusange,-3.5655,30.0230,2.0,SOS Medias Burundi,Other,08 February. An unidentified armed group kille...,1,1.549912e+09,BDI
6,3080370.0,108,BUR8581,8581.0,2019-02-07,2019.0,1.0,Violence against civilians,Rumonge Communal Militia (Burundi),NaN,...,Rumonge,-3.9736,29.4386,1.0,Local Source,Other,07 February. A mob of people attacked a man ac...,0,1.550526e+09,BDI
7,2998714.0,108,BUR8569,8569.0,2019-02-06,2019.0,2.0,Riots/Protests,Protesters (Burundi),NaN,...,Bujumbura,-3.3772,29.3580,1.0,SOS Medias Burundi,Other,06 February (on or before). Residents of Kinya...,0,1.549912e+09,BDI
8,3080367.0,108,BUR8578,8578.0,2019-02-06,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Bugabira,-2.5314,30.0706,1.0,Local Source,Other,06 February. Imbonerakure assaulted an elderly...,0,1.550526e+09,BDI
9,3080368.0,108,BUR8579,8579.0,2019-02-06,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Bubanza,-3.0804,29.3910,1.0,Local Source,Other,06 February. Imbonerakure assaulted a child wh...,0,1.550526e+09,BDI


In [64]:
burundi_dataset.head(5)

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,actor1,assoc_actor_1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,NaN,#country+code,#event+code,NaN,#date+occurred,NaN,NaN,#event+type,#group+name+first,#group+name+first+assoc,...,#loc+name,#geo+lat,#geo+lon,NaN,#meta+source,NaN,#description,#affected+killed,NaN,NaN
1,3080374.0,108,BUR8585,8585.0,2019-02-11,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Gitega,-3.4264,29.9308,1.0,SOS Medias Burundi,Other,"11 February. During the night, Imbonerakure br...",0,1.550526e+09,BDI
2,3080373.0,108,BUR8584,8584.0,2019-02-10,2019.0,1.0,Violence against civilians,CNDD-FDD-Imbonerakure: National Council for th...,NaN,...,Kaguhu,-2.8359,30.1756,1.0,Local Source,Other,10 February. Imbonerakure beat a man to death ...,1,1.550526e+09,BDI
3,3080371.0,108,BUR8582,8582.0,2019-02-09,2019.0,2.0,Violence against civilians,Unidentified Armed Group (Burundi),NaN,...,Carama,-3.3071,29.3813,2.0,Local Source,Other,09 February (on or before). The head of an uni...,1,1.550526e+09,BDI
4,3080372.0,108,BUR8583,8583.0,2019-02-09,2019.0,1.0,Violence against civilians,Rioters (Burundi),Vigilante Group (Burundi),...,Burimbi,-2.9479,29.4034,1.0,Local Source,Other,09 February. An unidentified group set fire to...,3,1.550526e+09,BDI
